In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from models import Net

net = gpu(Net())
print(net)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


from data_load import FacialKeypointsDataset

from data_load import Rescale, RandomCrop, Normalize, ToTensor




data_transform = None


assert(data_transform is not None), 'Define a data_transform'

In [ ]:
transformed_dataset = FacialKeypointsDataset(csv_file='data\\training_frames_keypoints.csv',
                                      root_dir='data\\training\\',
                                             transform=data_transform)

In [ ]:
print('Number of images: ', len(transformed_dataset))

# make sure the sample tensors are the expected size
for i in range(5):
    sample = transformed_dataset[i]
    print(i, sample['image'].size(), sample['keypoints'].size())
    
transformed_dataset[1]['image'].shape

In [ ]:
batch_size = 128

train_loader = DataLoader(transformed_dataset, 
                          batch_size=batch_size,
                          shuffle=True, 
                          num_workers=0)

In [ ]:
# create the test dataset
test_dataset = FacialKeypointsDataset(csv_file='data\\test_frames_keypoints.csv',
                                      root_dir='data\\test\\',
                                             transform=data_transform)

In [ ]:
# load test data in batches
batch_size = 128

test_loader = DataLoader(test_dataset, 
                          batch_size=batch_size,
                          shuffle=True, 
                          num_workers=0)

In [ ]:
# test the model on a batch of test images

def net_sample_output():
    
    # iterate through the test dataset
    for i, sample in enumerate(test_loader):
        
        # get sample data: images and ground truth keypoints
        images = sample['image']
        key_pts = sample['keypoints']

        # convert images to FloatTensors
        images = images.type(torch.cuda.FloatTensor)

        # forward pass to get net output
        output_pts = net(images)
        
        # reshape to batch_size x 68 x 2 pts
        output_pts = output_pts.view(output_pts.size()[0], 68, -1)
        
        # break after first image is tested
        if i == 0:
            return images, output_pts, key_pts

In [ ]:
test_images, test_outputs, gt_pts = net_sample_output()

# print out the dimensions of the data to see if they make sense
print(test_images.data.size())
print(test_outputs.data.size())
print(gt_pts.size())

In [ ]:
def show_all_keypoints(image, predicted_key_pts, gt_pts=None):
    """Show image with predicted keypoints"""
    # image is grayscale
    plt.imshow(image, cmap='gray')
    plt.scatter(predicted_key_pts[:, 0], predicted_key_pts[:, 1], s=20, marker='.', c='r')
    # plot ground truth points as green pts
    if gt_pts is not None:
        plt.scatter(gt_pts[:, 0], gt_pts[:, 1], s=20, marker='.', c='g')

In [ ]:

def visualize_output(test_images, test_outputs, gt_pts=None, batch_size=10):

    for i in range(batch_size):
        plt.figure(figsize=(20,10))
        ax = plt.subplot(1, batch_size, i+1)

       
        image = test_images[i].data   
        image = image.cpu().numpy()   # convert to numpy array from a Tensor
        image = np.transpose(image, (1, 2, 0))   # transpose to go from torch to numpy image

        
        predicted_key_pts = test_outputs[i].data
        predicted_key_pts = predicted_key_pts.cpu().numpy()
        
        predicted_key_pts = predicted_key_pts*50.0+100
        
        # plot ground truth points for comparison, if they exist
        ground_truth_pts = None
        if gt_pts is not None:
            ground_truth_pts = gt_pts[i]         
            ground_truth_pts = ground_truth_pts*50.0+100
        
        # call show_all_keypoints
        show_all_keypoints(np.squeeze(image), predicted_key_pts, ground_truth_pts)
            
        plt.axis('off')

    plt.show()
    
# call it
visualize_output(test_images, test_outputs, gt_pts)

In [ ]:
import torch.optim as optim

criterion = gpu(nn.MSELoss())

# ADAM optimizer
initial_lr=0.0001
optimizer = optim.Adam(net.parameters(), lr=initial_lr, amsgrad=True, weight_decay=0)



In [ ]:
def sqrt_lr_scheduler(optimizer, epoch, init_lr=initial_lr, steps=50):
    """Decay learning rate by the square root of the epoch."""
    if (epoch % steps) == 0:
        lr = init_lr / np.sqrt(epoch+1)
        print(f'Adjusted learning rate: {lr:.6f}')
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    return optimizer

In [ ]:
def train_net(n_epochs, initial_lr=initial_lr):
    global optimizer
    # prepare the net for training
    net.train()
    losses = []
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        #optimizer = sqrt_lr_scheduler(optimizer, epoch, initial_lr)

        # train on batches of data, assumes you already have train_loader
        for batch_i, data in enumerate(train_loader):
            # get the input images and their corresponding labels
            images = data['image']
            key_pts = data['keypoints']

            # flatten pts
            key_pts = key_pts.view(key_pts.size(0), -1)
            
            # wrap them in a torch Variable

            # convert variables to floats for regression loss
            key_pts = gpu(key_pts.type(torch.FloatTensor))
            images = gpu(images.type(torch.FloatTensor))

            # forward pass to get outputs
            output_pts = net(images)

            # calculate the loss between predicted and target keypoints
            loss = criterion(output_pts, key_pts)

            # zero the parameter (weight) gradients
            optimizer.zero_grad()
            
            # backward pass to calculate the weight gradients
            loss.backward()

            # update the weights
            optimizer.step()

            # print loss statistics
            running_loss += loss.data[0]
            if batch_i % 20 == 19:    # print every 20 batches
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, running_loss/1000))
                running_loss = 0.0
        losses.append(running_loss)

    print('Finished Training')
    return losses

In [ ]:
# train your network
n_epochs = 1100 

losses = train_net(n_epochs)

In [ ]:
losses = train_net(n_epochs)

In [ ]:
plt.figure()
plt.semilogy(losses)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('MSE');

In [ ]:
# get a sample of test data again
test_images, test_outputs, gt_pts = net_sample_output()

print(test_images.data.size())
print(test_outputs.data.size())
print(gt_pts.size())

In [ ]:
visualize_output(test_images, test_outputs)

In [ ]:

model_dir = ''
model_name = 'keypoints_model_1.pt'

# after training, save your model parameters in the dir 'saved_models'
torch.save(net.state_dict(), model_dir+model_name)
#torch.save(net, model_dir+model_name)